In [1]:
import os
import re
import time
import shutil
import pyodbc
import fnmatch
import numpy as np
import pandas as pd
import win32com.client
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
def generate_parts(target_values, num_parts, tolerance):
    parts_list = []
    for target_value in target_values:
        total_sum = target_value * num_parts
        while True:
            # Generate random parts
            parts = np.random.uniform(low=total_sum / num_parts * 0.9, high=total_sum / num_parts * 1.1, size=num_parts)
            # Ensure the sum is correct
            if np.abs(np.sum(parts) - total_sum) < tolerance:
                parts_list.append(parts)
                break
    return parts_list

# Find all relevant CSV files and process them
def process_csv_files(path):
    iri_dataframes = {}
    rutting_dataframes = {}

    for root, dirs, files in os.walk(path):
        # Find files
        iri_files = [f for f in files if f.endswith('.csv') and 'xw_iri_qgis' in f]
        rutting_files = [f for f in files if f.endswith('.csv') and 'xw_rutting' in f]
        
        # Process 'xw_iri_qgis' files
        for filename in iri_files:
            file_path = os.path.join(root, filename)
            iri_df = pd.read_csv(file_path, delimiter=';')
            iri_df.columns = iri_df.columns.str.strip()
            survey_code = filename.split('_')[3].split('.')[0]
            iri_df['survey_code'] = survey_code
            iri_df['iri'] = (iri_df['iri left (m/km)'] + iri_df['iri right (m/km)']) / 2        
            iri_df.drop(columns=['geometry'], errors='ignore', inplace=True)
            
            # Generate random values
            target_values = iri_df['iri']
            num_parts = 4
            tolerance = 0.3
            parts_list = generate_parts(target_values, num_parts, tolerance)

            # Expand DataFrame by repeating the rows
            iri_df = iri_df.loc[iri_df.index.repeat(num_parts)].reset_index(drop=True)
            iri_df['iri_lane'] = np.concatenate(parts_list)
            
            # Set initial event columns
            increment = 5 if fnmatch.fnmatch(filename, '*xw_iri_qgis*') else 5
            iri_df['chainage'] = iri_df.index * 5
            iri_df['event_start'] = range(0, len(iri_df) * increment, increment)
            iri_df['event_end'] = iri_df['event_start'] + increment
            iri_dataframes[filename] = iri_df
            
            # print(f"Updated {filename} into IRI DataFrame.")
        
        # Process 'xw_rutting' files
        for filename in rutting_files:
            file_path = os.path.join(root, filename)
            rut_df = pd.read_csv(file_path, delimiter=';')
            rut_df.columns = rut_df.columns.str.strip()
            if 'Unnamed: 5' in rut_df.columns:
                rut_df.drop(columns=['Unnamed: 5'], inplace=True, errors='ignore')
            else:
                pass
            increment = 5 if fnmatch.fnmatch(filename, '*xw_rutting*') else 5
            rut_df['event_start'] = range(0, len(rut_df) * increment, increment)
            rut_df['event_end'] = rut_df['event_start'] + increment
            
            survey_code = filename.split('_')[2].split('.')[0]
            rut_df['index'] = rut_df.index * 25 // 5
            rut_df.set_index('index', inplace=True)
            rut_df['survey_code'] = survey_code
            rut_df['rut_point_x'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[1]))
            rut_df['rut_point_y'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[0]))
            rut_df['rut_point_x'].fillna(0, inplace=True)
            rut_df['rut_point_y'].fillna(0, inplace=True)
        
            rut_df.rename(columns={'left rutting height': 'left_rutting', 'right rutting height': 'right_rutting', 'average height': 'avg_rutting'}, inplace=True)
            rut_df.drop(columns=['qgis_shape'], inplace=True)
            rutting_dataframes[filename] = rut_df

            # print(f"Updated {filename} into Rutting DataFrame.")

    print(f"✅ Finished processing: .CSV files.")
    return iri_dataframes, rutting_dataframes

# Perform the left join on xw_rutting and xw_iri_qgis
def left_join_dataframes(df_rutting, df_iri):
    joined_df = pd.merge(df_rutting, df_iri, how='left', on=['survey_code'], suffixes=('_rutting', '_iri'))
    joined_df = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end', 'survey_code'], suffixes=('_rutting', '_iri'))
    return joined_df

# Perform jpg file and frame number
def get_jpg_names_and_nums(directory):
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)

    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace(r'_(\d+)', lambda m: f"RUN{int(m.group(1)):02d}", regex=True)
    return jpg_files, frame_df

# Perform fainal data frame
def process_fainal_df(output_dir):
    jpg_files, frame_df = get_jpg_names_and_nums(output_dir)
    iri_dataframes, rutting_dataframes = process_csv_files(output_dir)

    joined_dataframes = {}
    for rutting_file in rutting_dataframes:
        for iri_file in iri_dataframes:
            if 'xw_rutting' in rutting_file and 'xw_iri_qgis' in iri_file:
                joined_df = left_join_dataframes(rutting_dataframes[rutting_file], iri_dataframes[iri_file])

                # Group by survey_code and calculate min and max chainage
                grouped_df = joined_df.groupby('survey_code').agg(
                    max_chainage=('chainage', 'max'),
                    min_chainage=('chainage', 'min')
                ).reset_index()

                joined_df = pd.merge(joined_df, grouped_df, on='survey_code', how='left')
                joined_df = pd.merge(joined_df, frame_df, on='survey_code', how='left')
                joined_df['frame_num'] = joined_df.groupby(['survey_code', 'pic_count']).cumcount() + 1
                joined_df['chainage_pic'] = round(joined_df['max_chainage'] / joined_df['pic_count'] * joined_df['frame_num'],0)
                joined_dataframes[f"{rutting_file}_{iri_file}"] = joined_df
                
    final_df = pd.concat(joined_dataframes.values(), ignore_index=True)
    
    selected_columns = [
        'left_rutting', 'right_rutting', 'avg_rutting', 'event_start', 'event_end', 'survey_code',
        'rut_point_x', 'rut_point_y', 'Date', 'iri left (m/km)', 'iri right (m/km)', 'iri', 'iri_lane', 
        'chainage', 'max_chainage', 'min_chainage', 'pic_count', 'frame_num', 'chainage_pic'
    ]
    
    selected_columns = [col for col in selected_columns if col in final_df.columns]
    final_df = final_df[final_df['iri'].notnull()][selected_columns]
    return final_df

In [ ]:
# Find all relevant CSV files and process them
def process_csv_files(path):
    iri_dataframes = {}
    rutting_dataframes = {}

    for root, dirs, files in os.walk(path):
        # Find files
        iri_files = [f for f in files if f.endswith('.csv') and 'xw_iri_qgis' in f]
        rutting_files = [f for f in files if f.endswith('.csv') and 'xw_rutting' in f]
        
        # Process 'xw_iri_qgis' files
        for filename in iri_files:
            file_path = os.path.join(root, filename)
            iri_df = pd.read_csv(file_path, delimiter=';')
            iri_df.columns = iri_df.columns.str.strip()
            survey_code = filename.split('_')[3].split('.')[0]
            iri_df['survey_code'] = survey_code
            iri_df['iri'] = (iri_df['iri left (m/km)'] + iri_df['iri right (m/km)']) / 2        
            iri_df.drop(columns=['geometry'], errors='ignore', inplace=True)
            
            # Generate random values
            target_values = iri_df['iri']
            num_parts = 4
            tolerance = 0.3
            parts_list = generate_parts(target_values, num_parts, tolerance)

            # Expand DataFrame by repeating the rows
            iri_df = iri_df.loc[iri_df.index.repeat(num_parts)].reset_index(drop=True)
            iri_df['iri_lane'] = np.concatenate(parts_list)
            
            # Set initial event columns
            increment = 5 if fnmatch.fnmatch(filename, '*xw_iri_qgis*') else 5
            iri_df['chainage'] = iri_df.index * 5
            iri_df['event_start'] = range(0, len(iri_df) * increment, increment)
            iri_df['event_end'] = iri_df['event_start'] + increment
            iri_dataframes[filename] = iri_df
            
            # print(f"Updated {filename} into IRI DataFrame.")
        
        # Process 'xw_rutting' files
        for filename in rutting_files:
            file_path = os.path.join(root, filename)
            rut_df = pd.read_csv(file_path, delimiter=';')
            rut_df.columns = rut_df.columns.str.strip()
            if 'Unnamed: 5' in rut_df.columns:
                rut_df.drop(columns=['Unnamed: 5'], inplace=True, errors='ignore')
            else:
                pass
            increment = 5 if fnmatch.fnmatch(filename, '*xw_rutting*') else 5
            rut_df['event_start'] = range(0, len(rut_df) * increment, increment)
            rut_df['event_end'] = rut_df['event_start'] + increment
            
            survey_code = filename.split('_')[2].split('.')[0]
            rut_df['index'] = rut_df.index * 25 // 5
            rut_df.set_index('index', inplace=True)
            rut_df['survey_code'] = survey_code
            rut_df['rut_point_x'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[1]))
            rut_df['rut_point_y'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[0]))
            rut_df['rut_point_x'].fillna(0, inplace=True)
            rut_df['rut_point_y'].fillna(0, inplace=True)
        
            rut_df.rename(columns={'left rutting height': 'left_rutting', 'right rutting height': 'right_rutting', 'average height': 'avg_rutting'}, inplace=True)
            rut_df.drop(columns=['qgis_shape'], inplace=True)
            rutting_dataframes[filename] = rut_df

            # print(f"Updated {filename} into Rutting DataFrame.")

    print(f"✅ Finished processing: .CSV files.")
    return iri_dataframes, rutting_dataframes

In [ ]:
def get_jpg_names_and_nums(directory):
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)

    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace(r'_(\d+)', lambda m: f"RUN{int(m.group(1)):02d}", regex=True)
    return jpg_files, frame_df

In [17]:
base_dir = r"D:\xenomatix"
input_dir = os.path.join(base_dir, "input")
output_dir = os.path.join(base_dir, "output")

In [43]:
jpg_files, frame_df = get_jpg_names_and_nums(output_dir)

In [49]:
jpg_files = pd.DataFrame(jpg_files)
jpg_files.columns = ['filename']
jpg_files.tail(10)

,filename
847,20240726_3-ROW-0-00848.jpg
848,20240726_3-ROW-0-00849.jpg
849,20240726_3-ROW-0-00850.jpg
850,20240726_3-ROW-0-00851.jpg
851,20240726_3-ROW-0-00852.jpg
852,20240726_3-ROW-0-00853.jpg
853,20240726_3-ROW-0-00854.jpg
854,20240726_3-ROW-0-00855.jpg
855,20240726_3-ROW-0-00856.jpg
856,20240726_3-ROW-0-00857.jpg


In [18]:
iri_dataframes, rutting_dataframes = process_csv_files(output_dir)

✅ Finished processing: .CSV files.


In [25]:
iri_dataframes['xw_iri_qgis_20240726RUN01.csv'].head(20)

,Date,iri left (m/km),iri Std left (m/km),iri right (m/km),iri Std right (m/km),worst iri (m/km),iri difference (m/km),survey_code,iri,iri_lane,chainage,event_start,event_end
0,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,20240726RUN01,3.754977,3.530337,0,0,5
1,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,20240726RUN01,3.754977,3.860193,5,5,10
2,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,20240726RUN01,3.754977,3.908493,10,10,15
3,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,20240726RUN01,3.754977,3.502770,15,15,20
4,26/07/2024,4.101019,3.490062,4.845301,3.479725,4.845301,0.744282,20240726RUN01,4.473160,4.565608,20,20,25
5,26/07/2024,4.101019,3.490062,4.845301,3.479725,4.845301,0.744282,20240726RUN01,4.473160,4.840924,25,25,30
6,26/07/2024,4.101019,3.490062,4.845301,3.479725,4.845301,0.744282,20240726RUN01,4.473160,4.149929,30,30,35
7,26/07/2024,4.101019,3.490062,4.845301,3.479725,4.845301,0.744282,20240726RUN01,4.473160,4.219810,35,35,40
8,26/07/2024,4.771488,3.811178,5.989904,3.969238,5.989904,1.218416,20240726RUN01,5.380696,5.104996,40,40,45
9,26/07/2024,4.771488,3.811178,5.989904,3.969238,5.989904,1.218416,20240726RUN01,5.380696,5.681348,45,45,50


In [28]:
rutting_dataframes['xw_rutting_20240726RUN01.csv'].head(20)

,#Date,left_rutting,right_rutting,avg_rutting,event_start,event_end,survey_code,rut_point_x,rut_point_y
index,,,,,,,,,
0,26/07/2024,3.46,5.29,4.38,0,5,20240726RUN01,13.870717,100.451832
5,26/07/2024,1.49,4.30,2.90,5,10,20240726RUN01,13.870672,100.451835
10,26/07/2024,2.35,0.42,1.38,10,15,20240726RUN01,13.870627,100.451839
15,26/07/2024,1.96,0.55,1.25,15,20,20240726RUN01,13.870582,100.451842
20,26/07/2024,1.69,0.71,1.20,20,25,20240726RUN01,13.870537,100.451846
25,26/07/2024,0.14,4.70,2.42,25,30,20240726RUN01,13.870492,100.451849
30,26/07/2024,0.07,5.80,2.94,30,35,20240726RUN01,13.870446,100.451852
35,26/07/2024,2.83,2.07,2.45,35,40,20240726RUN01,13.870401,100.451855
40,26/07/2024,0.48,4.70,2.59,40,45,20240726RUN01,13.870356,100.451858


In [19]:
final_df = process_fainal_df(output_dir)

✅ Finished processing: .CSV files.


In [26]:
final_df[final_df['survey_code'] == '20240726RUN01'].head(20)

,left_rutting,right_rutting,avg_rutting,event_start,event_end,survey_code,rut_point_x,rut_point_y,Date,iri left (m/km),iri right (m/km),iri,iri_lane,chainage,max_chainage,min_chainage,pic_count,frame_num,chainage_pic
0,3.46,5.29,4.38,0,5,20240726RUN01,13.870717,100.451832,26/07/2024,3.824703,3.685251,3.754977,3.572767,0.0,1635.0,0.0,247,1,7.0
1,1.49,4.30,2.90,5,10,20240726RUN01,13.870672,100.451835,26/07/2024,3.824703,3.685251,3.754977,3.929628,5.0,1635.0,0.0,247,2,13.0
2,2.35,0.42,1.38,10,15,20240726RUN01,13.870627,100.451839,26/07/2024,3.824703,3.685251,3.754977,3.631014,10.0,1635.0,0.0,247,3,20.0
3,1.96,0.55,1.25,15,20,20240726RUN01,13.870582,100.451842,26/07/2024,3.824703,3.685251,3.754977,3.651102,15.0,1635.0,0.0,247,4,26.0
4,1.69,0.71,1.20,20,25,20240726RUN01,13.870537,100.451846,26/07/2024,4.101019,4.845301,4.473160,4.888263,20.0,1635.0,0.0,247,5,33.0
5,0.14,4.70,2.42,25,30,20240726RUN01,13.870492,100.451849,26/07/2024,4.101019,4.845301,4.473160,4.507635,25.0,1635.0,0.0,247,6,40.0
6,0.07,5.80,2.94,30,35,20240726RUN01,13.870446,100.451852,26/07/2024,4.101019,4.845301,4.473160,4.251843,30.0,1635.0,0.0,247,7,46.0
7,2.83,2.07,2.45,35,40,20240726RUN01,13.870401,100.451855,26/07/2024,4.101019,4.845301,4.473160,4.509344,35.0,1635.0,0.0,247,8,53.0
8,0.48,4.70,2.59,40,45,20240726RUN01,13.870356,100.451858,26/07/2024,4.771488,5.989904,5.380696,5.464147,40.0,1635.0,0.0,247,9,60.0
9,1.22,4.04,2.63,45,50,20240726RUN01,13.870311,100.451861,26/07/2024,4.771488,5.989904,5.380696,4.929626,45.0,1635.0,0.0,247,10,66.0


In [ ]:
# join chainage_pic >= chainage and chainage_pic <= chainage + 5

In [4]:
import pandas as pd
import os
import fnmatch

# Define the path to your directories
directory_path = r"D:\xenomatix\output\survey_data_20240726\Output\20240726RUN02"
jpg_directory_path = r"D:\xenomatix\output\survey_data_20240726\PAVE\20240726_2\PAVE-0"

# Dictionary to store DataFrames
dataframes = {}

# Function to create new rows based on intervals
def create_interval_rows(df, start_col='event_start', end_col='event_end', interval_size=5):
    new_rows = []
    
    for _, row in df.iterrows():
        start = row[start_col]
        end = row[end_col]
        
        while start < end:
            new_row = row.copy()
            new_row[start_col] = start
            new_row[end_col] = start + interval_size
            new_rows.append(new_row)
            start += interval_size
    
    return pd.DataFrame(new_rows)

# Function to get .jpg file names
def get_jpg_filenames(directory):
    jpg_filenames = [filename for filename in os.listdir(directory) if filename.endswith(".jpg")]
    return jpg_filenames

# Function to extract numeric part from .jpg filenames
def extract_numeric_from_filenames(filenames):
    numeric_values = [int(fname.split('-')[-1].split('.jpg')[0]) for fname in filenames]
    return numeric_values

# Get the .jpg file names
jpg_filenames = get_jpg_filenames(jpg_directory_path)
frame_numbers = extract_numeric_from_filenames(jpg_filenames)

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory_path, filename)
        # Read the CSV file into a DataFrame with semicolon delimiter
        df = pd.read_csv(file_path, delimiter=';')
        
        # Determine the increment values for event_start and event_end
        if fnmatch.fnmatch(filename, '*xw_iri_qgis*'):
            increment = 20
        else:
            increment = 5
        
        # Initialize event_start and event_end columns
        df['event_start'] = 0
        df['event_end'] = increment

        # Set the initial values
        event_start = 0
        event_end = increment

        # Loop through the DataFrame and update event_start and event_end
        for i in range(len(df)):
            df.at[i, 'event_start'] = event_start
            df.at[i, 'event_end'] = event_end
            event_start = event_end
            event_end += increment

        # Remove or replace newlines in 'geometry' column if it exists
        if 'geometry' in df.columns:
            df['geometry'] = df['geometry'].str.replace('\n', ' ', regex=False)

        # Create new rows based on intervals
        new_df = create_interval_rows(df, interval_size=5)
        
        # Store the new DataFrame in the dictionary with the filename as the key
        dataframes[filename] = new_df

        print(f"Processed and updated {filename} into DataFrame.")

# Perform the left join on xw_rutting and xw_iri_qgis
def left_join_dataframes(df_rutting, df_iri):
    joined_df = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end'], suffixes=('_rutting', '_iri'))
    return joined_df

def add_frame_num_to_joined_df(joined_df, derived_values, frame_numbers):
    # Create new columns 'frame_num_ch' and 'frame_num' initialized with NaN
    joined_df['frame_num_ch'] = pd.NA
    joined_df['frame_num'] = pd.NA

    # Create a DataFrame to map derived values to frame numbers
    derived_to_frame_mapping = pd.DataFrame({
        'frame_num_ch': derived_values,
        'frame_num': frame_numbers
    })

    # Additional logic to update frame_num_ch and frame_num based on derived_values
    for i, frame_num_ch in enumerate(derived_values):
        mask = (joined_df['event_start'] <= frame_num_ch) & (joined_df['event_end'] > frame_num_ch)
        joined_df.loc[mask, 'frame_num_ch'] = frame_num_ch
        joined_df.loc[mask, 'frame_num'] = frame_numbers[i]

    return joined_df


# Perform the left join and store the result
joined_dataframes = {}
for rutting_file in dataframes:
    if fnmatch.fnmatch(rutting_file, '*xw_rutting*'):
        for iri_file in dataframes:
            if fnmatch.fnmatch(iri_file, '*xw_iri_qgis*'):
                joined_df = left_join_dataframes(dataframes[rutting_file], dataframes[iri_file])
                
                # Get the maximum event_end
                max_event_start = joined_df['event_start'].max()
                
                # Process the .jpg filenames to extract the derived values
                derived_values = [round((max_event_start * num) / max(frame_numbers)) for num in frame_numbers]
                print('max_event_end = ', max_event_start)
                print('frame_numbers = ', max(frame_numbers))
                print("Derived Values:", derived_values)
                print("Count of derived_values:", len(derived_values))
                print("Frame Numbers:", frame_numbers)
                print("Count of frame_numbers:", len(frame_numbers))
                
                # Add frame_num and frame_num_ch to the joined DataFrame
                joined_df = add_frame_num_to_joined_df(joined_df, derived_values, frame_numbers)
               
                # Store the joined DataFrame in the dictionary
                joined_dataframes[f"{rutting_file}_{iri_file}"] = joined_df

# Print the joined DataFrames with frame numbers
for key in joined_dataframes:
    print(f"\nJoined DataFrame with frame numbers for {key}:")
    print(joined_dataframes[key].to_string(index=False))

Processed and updated xw_iri_qgis_20240726RUN02.csv into DataFrame.
Processed and updated xw_rutting_20240726RUN02.csv into DataFrame.
max_event_end =  6180
frame_numbers =  880
Derived Values: [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119, 126, 133, 140, 147, 154, 162, 169, 176, 183, 190, 197, 204, 211, 218, 225, 232, 239, 246, 253, 260, 267, 274, 281, 288, 295, 302, 309, 316, 323, 330, 337, 344, 351, 358, 365, 372, 379, 386, 393, 400, 407, 414, 421, 428, 435, 442, 449, 456, 464, 471, 478, 485, 492, 499, 506, 513, 520, 527, 534, 541, 548, 555, 562, 569, 576, 583, 590, 597, 604, 611, 618, 625, 632, 639, 646, 653, 660, 667, 674, 681, 688, 695, 702, 709, 716, 723, 730, 737, 744, 751, 758, 765, 772, 780, 787, 794, 801, 808, 815, 822, 829, 836, 843, 850, 857, 864, 871, 878, 885, 892, 899, 906, 913, 920, 927, 934, 941, 948, 955, 962, 969, 976, 983, 990, 997, 1004, 1011, 1018, 1025, 1032, 1039, 1046, 1053, 1060, 1067, 1074, 1082, 1089, 1096, 1103, 1110, 1117, 1124, 11

In [3]:
dataframes

{'xw_iri_qgis_20240726RUN02.csv':            Date   iri left (m/km)   iri Std left (m/km)   iri right (m/km)  \
 0    26/07/2024          3.865407              2.804227           3.482863   
 0    26/07/2024          3.865407              2.804227           3.482863   
 0    26/07/2024          3.865407              2.804227           3.482863   
 0    26/07/2024          3.865407              2.804227           3.482863   
 1    26/07/2024          3.547611              2.251651           3.629724   
 ..          ...               ...                   ...                ...   
 306  26/07/2024          4.782251              4.015040           5.930027   
 307  26/07/2024          4.743412              4.057893           7.756645   
 307  26/07/2024          4.743412              4.057893           7.756645   
 307  26/07/2024          4.743412              4.057893           7.756645   
 307  26/07/2024          4.743412              4.057893           7.756645   
 
       iri Std ri

ใช้แค่รูปจาก frame_num แต่ chainage ใช้ 0 5 ที่ row มา